# QPM : Assignement 7

## Librairies

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

## Functions

In [2]:
'''
Function: annual_mean_returns ; annual_volatility ; sharpe_ratio

Returns the annual mean, return and sharpe ratio of each stocks.

Input: - df: DataFrame (float) ; DataFrame of time series of close price of each stocks.

Output: DataFrame (float) ; 1 column, indexed by the stocks tickers.
'''

def annual_mean_return(df) :
    return df.mean() * 12

def annual_volatility(df) :
    return df.std() * np.sqrt(12)

def sharpe_ratio(df) :
    return annual_mean_return(df) / annual_volatility(df)

In [3]:
def objective(weights, mean_returns, cov_matrix, gamma):
    """
    Objective function, which we want to minimize in order to build an optimal allocation.
    Input: 
        - weights: list (floats), initial weights for the allocation ;
        - mean_returns: list (floats), mean returns of each factor ;
        - cov_matrix: ndarray (floats), covariance matrix of the factors returns ;
        - gamma: float, risk aversion.

    Output: utility: float, value of the utility function given the arguments.
    """
    complete_weights = np.concatenate([[1],weights])
    portfolio_return = np.dot(complete_weights, mean_returns)
    portfolio_volatility = np.sqrt(np.dot(complete_weights.T, np.dot(cov_matrix, complete_weights)))
    utility = portfolio_return - 0.5 * gamma * portfolio_volatility ** 2
    return -utility

**Q7.1 Explain why one might expect these nine factors to be related to stock
returns.**

**Market**
The Market factor, often represented by the return of a broad market index such as the S&P 500, is considered relevant to stock returns because it captures systematic risk associated with overall market movements, influencing the performance of individual stocks as they tend to respond to broader market trends and economic conditions.

**SMB**
The SMB (Small Minus Big) factor is relevant to stock returns because it reflects the historical excess returns of small-cap stocks over large-cap stocks, indicating that smaller companies have exhibited a tendency to outperform their larger counterparts, potentially due to risk factors associated with size and market conditions.

**HML**
The HML (High Minus Low) factor is pertinent to stock returns as it captures the historical performance difference between portfolios of stocks with high book-to-market ratios (value stocks) and low book-to-market ratios (growth stocks), suggesting that value stocks have demonstrated a tendency to outperform growth stocks over time, potentially due to risk and market mispricing factors.

**RMW**
The RMW (Robust Minus Weak) factor is relevant to stock returns as it represents the historical performance gap between firms with robust profitability and those with weak profitability. This factor suggests that stocks of companies with stronger profitability have exhibited a tendency to outperform those with weaker profitability, indicating the significance of profitability as a factor influencing stock returns.

**CMA**
The CMA (Conservative Minus Aggressive) factor is pertinent to stock returns as it reflects the historical performance difference between companies with conservative accounting practices and those with aggressive accounting practices. This factor suggests that stocks of companies with conservative accounting methods have shown a tendency to outperform those with aggressive practices, indicating the impact of accounting conservatism on stock returns.

**UMD** 
The UMD (Up-minus-Down) factor is relevant to stock returns as it represents the historical performance spread between past winners and past losers, indicating that stocks with strong recent performance (winners) have tended to continue outperforming stocks with weak recent performance (losers), reflecting the presence of short-term momentum effects in the market.

**ROE** 
The ROE (Return on Equity) factor is relevant to stock returns as it reflects the historical performance difference between companies with high and low returns on equity. This factor suggests that stocks of companies with higher ROE have demonstrated a tendency to outperform stocks of companies with lower ROE, indicating the importance of profitability metrics in influencing stock returns.

**IA**
The investment factor is relevant to stock returns because it captures the historical performance difference between companies that invest conservatively (low investment) and those that invest more aggressively (high investment). This factor suggests that stocks of companies with lower levels of investment relative to their assets have tended to outperform stocks of companies with higher investment levels, indicating a link between investment strategies and stock returns.

**BAB**
The betting-against-beta (BAB) factor is relevant to stock returns as it reflects the historical performance difference between portfolios of stocks with low and high market beta. This factor suggests that, contrary to the Capital Asset Pricing Model (CAPM), low-beta stocks have exhibited a tendency to outperform high-beta stocks, indicating a phenomenon where investors may receive higher returns by betting against beta.

**Q7.2 Find the optimal θ vector for a mean-variance investor with risk aversion of γ = 5 if the investor can invest in only these nine factors**  
Let's import the data of the nine factors

In [ ]:
#Import data
factor_data = pd.read_excel("QPM-FactorsData.xlsx", index_col = 0, parse_dates=True)
#Set index type to DateTime
factor_data.index = pd.to_datetime(factor_data.index, format='%Y%m')
F_factor_data = factor_data.copy()
#The 'Market' data is used as a benchmark.
F_factor_data.pop('Market')

factor_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 647 entries, 1967-02-01 to 2020-12-01
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Market  647 non-null    float64
 1   SMB     647 non-null    float64
 2   HML     647 non-null    float64
 3   RMW     647 non-null    float64
 4   CMA     647 non-null    float64
 5   UMD     647 non-null    float64
 6   ROE     647 non-null    float64
 7   IA      647 non-null    float64
 8   BAB     647 non-null    float64
dtypes: float64(9)
memory usage: 50.5 KB


/var/folders/lr/7kzpljcn0zl52pt8zpscgfj40000gn/T/ipykernel_5720/1865887156.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  factor_data = pd.read_excel("QPM-FactorsData.xlsx", index_col = 0, parse_dates=True)


In [5]:
#Risk aversion value
gamma = 5
#Compute mean returns and covariance
mean_returns = factor_data.mean()
cov_matrix = factor_data.cov()
mean_returns_bench = factor_data['Market'].mean()
cov_matrix_bench = factor_data['Market'].var()
F_mean_returns = F_factor_data.mean()
F_cov_matrix = F_factor_data.cov()

We perform the optimistion to find optimal weights, defining the constraint that the sum of the weights equal 1 and assuming weights can be negative for short-selling constraints.
We minimise the negative of the utlity to maximise the utility itself

In [ ]:
#initial weights: equally-weighted portfolio.
initial_weights = np.ones(len(F_mean_returns)) / len(F_mean_returns)
#Change the weights so it maximizes the utility function / minimizes the objective function (= -utility)
result = minimize(objective, initial_weights, args=(mean_returns, cov_matrix, gamma),
                  method='SLSQP', 
                  constraints=({'type': 'eq', 'fun': lambda weights: np.sum(weights)}), #Factor pfo has a sum weight of 0. 
                  bounds=tuple((None, None) for _ in range(len(F_mean_returns))))

[0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]


In [7]:
optimal_weights = np.concatenate([[1],result.x])
print("The optimal vector for the benchmark and the 8 factors is given by:", optimal_weights, sep = "\n")

The optimal vector for the benchmark and the 8 factors is given by:
[ 1.         -0.80362166 -0.58679178 -1.30616889  0.05176532  0.06899556
  0.71009105  0.6612912   1.2044392 ]


**Q7.3 Find the Sharpe ratio for each of the nine factors and compare it to that of the parametric portfolio you have identified in the previous question.**

All factors are considered excess returns so no need to remove the risk-free rate.

In [8]:
sharpe_ratio_factors = sharpe_ratio(factor_data)
print("The Sharpe ratio for the benchmark and the 8 factors is", sharpe_ratio_factors,sep = "\n" )

The Sharpe ratio for the benchmark and the 8 factors is
Market    0.432904
SMB       0.197861
HML       0.277910
RMW       0.420687
CMA       0.477857
UMD       0.507284
ROE       0.684304
IA        0.626422
BAB       0.899567
dtype: float64


In [9]:
optimal_return = np.dot(optimal_weights, mean_returns)
optimal_volatility = np.sqrt(np.dot(optimal_weights.T, np.dot(cov_matrix, optimal_weights)))
sharpe_ratio_parametric = optimal_return/optimal_volatility
print("The Sharpe ratio of the parametric portfolio is", sharpe_ratio_parametric)

The Sharpe ratio of the parametric portfolio is 0.2679461669564063


**Q7.4 Having obtained the optimal θ vector, please explain how one would
obtain the optimal portfolio weights for each of the Nt = 2000 assets that
are used to form each of the nine factors.**

To obtain the optimal portfolio weights for each of the 2000 assets that are used to form each of the nine factors, we can use the concept of factor loadings, representing the sensitivity of an individual asset's return to changes in a particular factor. In the context of the mean-variance optimization framework, the optimal weight for an asset in the portfolio can be determined based on its factor loadings and the optimal weights for the factors.

For any individual stock i, the optimal weight would be given by: (θ * Fi)/(sum of F).